In [9]:
import pandas as pd
from glob import glob

In [10]:
# Считываем все файлы с депозитами из папки, и делаем 1 таблицу
path = r'dep_folder'
all_files = glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep=';', engine='python')
    li.append(df)
data_dep = pd.concat(li, axis=0, ignore_index=True)
data_dep['Cумма депозитов'] = data_dep['Cумма депозитов'].str.replace(',', '.', regex=True)
data_dep['Cумма депозитов'] =  pd.to_numeric(data_dep['Cумма депозитов'])
data_dep

,ID игрока,День,Название валюты,Cумма депозитов
0,4,02.03.2020,UAH,10.000000
1,4,02.03.2020,UAH,0.000000
2,5,02.03.2020,UAH,75.000000
3,5,02.03.2020,UAH,50.000000
4,6,02.03.2020,UAH,1275.000000
...,...,...,...,...
229,6,04.03.2020,UAH,26.883333
230,7,04.03.2020,UAH,333.333333
231,21,04.03.2020,RUB,11.666667
232,22,04.03.2020,RUB,1633.333333


In [11]:
# гуппируем и берём наибольшее значение с колонки Сумма деп.
data_dep_gr = data_dep.groupby(['ID игрока', 'День', 'Название валюты'])['Cумма депозитов'].agg(Max_deposit = 'max')
data_dep_gr = pd.DataFrame(data_dep_gr).reset_index()
# берём только нужные валюты
data_dep_gr = data_dep_gr[data_dep_gr['Название валюты'].isin(['UAH', 'USD', 'RUB', 'EUR'])]
data_dep_gr

,ID игрока,День,Название валюты,Max_deposit
0,1,03.03.2020,UAH,5000.000000
1,2,03.03.2020,UAH,8450.000000
2,3,03.03.2020,UAH,5360.000000
3,4,02.03.2020,UAH,2500.000000
4,4,03.03.2020,UAH,1255.000000
5,5,02.03.2020,UAH,4225.000000
6,5,03.03.2020,UAH,49.000000
7,6,02.03.2020,UAH,2680.000000
8,6,03.03.2020,UAH,660.000000
9,6,04.03.2020,UAH,1666.666667


In [12]:
# функция которая посчитает к-во спинов
def free_spins_calc(df):
    if df['Название валюты'] == 'UAH':
        if 10000 > df['Max_deposit'] >= 1000:
            return 10
        elif df['Max_deposit'] >= 10000:
            return 50
        return 0
    if df['Название валюты']in(['EUR', 'USD']):
        if 1000 > df['Max_deposit'] >= 500:
            return 10
        elif df['Max_deposit'] >= 1000:
            return 50
        return 0
    if df['Название валюты'] == 'RUB':
        if 5000 > df['Max_deposit'] >= 1000:
            return 10
        elif df['Max_deposit'] >= 5000:
            return 50
        return 0

In [13]:
# применяем ф-ю
data_dep_gr['Free Spins'] = data_dep_gr.apply(lambda row: free_spins_calc(row), axis=1)

In [14]:
# убираем нули
res = data_dep_gr[data_dep_gr['Free Spins'] != 0].sort_values(by=['День','Название валюты','ID игрока']).reset_index(drop=True)
res

,ID игрока,День,Название валюты,Max_deposit,Free Spins
0,19,02.03.2020,EUR,1725.000000,50
1,20,02.03.2020,EUR,2450.000000,50
2,4,02.03.2020,UAH,2500.000000,10
3,5,02.03.2020,UAH,4225.000000,10
4,6,02.03.2020,UAH,2680.000000,10
5,10,02.03.2020,UAH,1325.000000,10
6,11,02.03.2020,UAH,2565.000000,10
7,15,02.03.2020,UAH,1793.500000,10
8,14,02.03.2020,USD,1000.000000,50
9,17,03.03.2020,EUR,4900.000000,50


In [15]:
# делим таблицу по дням и сохраняем
sep = res['День'].unique()
for i in sep:
    res[res['День'] == i].to_excel(f"{i}_FreeSpins.xlsx", index=False)
#     res[res['День'] == i].to_csv(f"{i}_FreeSpins.csv", index=False) эта строка сдеает csv-шки